<a href="https://colab.research.google.com/github/AndreBJusti/Proten/blob/main/Proten.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ezdxf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 29.1 MB/s eta 0:00:00


In [ ]:
import ezdxf
from ezdxf.gfxattribs import GfxAttribs
from ezdxf.enums import TextEntityAlignment

In [ ]:
from math import sqrt, tan, radians, sin, cos, atan
import pandas as pd

# To do:

- Deixar o usuério selecionar se ele quer usar a laje superior ou inferior como referência;
- Desenhar o perfil longitudinal da viga certo, inclusive com a variação parabólica de um BS;
- Calcular o raio das parábolas e verificar se atende ao raio mínimo;


- Para os cortes, eu tenho uma função que pega as coordenadas dos cabos para cada plano e cria um dicionário com as chaves sendo o nome do cabo e os valores uma lista de lista com a posição do corte e o valor x, ou z
- Tenho uma função que pega uma lista de dicionários em que cada dicionário é a posição de um corte e cada valor do dicionário é um cabo naquele corte com sua respectiva coordenada x, y
- Falta pegar a saida da primeira função e transformar no formato da entrada da segunda função




- Crirar uma interface (colocar na opção de desenho, quais desenhos quer criar)
- Importar coordenadas de um csv
- Pedir vão para saber os vigésimos
- Desenhar seção e cortes
- Pedir e tratar coordenadas da viga
- Alma inclinada (distancia da lage e corrige com o ângulo informado)
- Alterar para Orientado a Objeto
- Exportar coordenadas para csv e salvar para depois poder alterar

# Como está organizado:

- Dicionário para os cabos (sendo: chave = nome do cabo, valor = lista de coordenadas(lista de listas, com angulos e tipos))
- Lista das coordenadas da seção (já pode ser lista de tuplas)
-

# Cabos entrada

In [ ]:
# Pede o número de cabos, chama as funções para pedir as coordenadas e cria o dicionário

def entrada_dados():
  n_cabos = int(input("Número de cabos: "))
  cabos_planta = {}
  cabos_elevacao = {}
  for i in range(n_cabos):
    nome_cabo = "C." + str(i+1)
    cabos_planta[nome_cabo] = coordenadas_planta(i+1)
    cabos_elevacao[nome_cabo] = coordenadas_elevacao(i+1)
  return cabos_planta, cabos_elevacao # Ver se deixa essa variável local ou global


In [ ]:
# Pede as coordenadas

# Será substituida pela interface

def coordenadas_planta(cabo):
  # Pede as coordendas do traçado em planta
  lista_planta = []

  trechos = int(input(f'Número de trechos em planta do cabo {cabo}: '))

  for i in range(trechos+1):
    print(f'{i+1}º Ponto: ')
    x = float(input("x: "))
    y = float(input("y: "))
    angulo = float(input("Ângulo: "))
    xinf = float(input("x inf: "))
    tipo = str(input("r, p ou pi: "))
    lista_planta.append([x, y, angulo, xinf, tipo])

  return lista_planta

def coordenadas_elevacao(cabo):
  # Pede as coordenadas do traçado em elevação
  lista_elevacao = []

  trechos = int(input(f'Número de trechos em elevação do cabo {cabo}: '))

  for i in range(trechos+1):
    print(f'{i+1}º Ponto: ')
    x = float(input("x: "))
    z = float(input("z: "))
    angulo = float(input("Ângulo: "))
    xinf = float(input("x inf: "))
    tipo = str(input("r, p ou pi: "))
    lista_elevacao.append([x, z, angulo, xinf, tipo])

  return lista_elevacao


# Cabos desenho

In [ ]:
# Acrescenta trecho reto de 1m nas pontas

def sobras(pi, pf):
  # Pega a inclinação do primeiro ponto, com o ângulo calcula o x e o y novo para q a hipotenusa tenha 1m
  # p = [x, y, ang, -, 'tipo']
  xi = pi[0]
  yi = pi[1]
  angi = radians(pi[2])

  dxi = cos(angi)
  dyi = sin(angi)

  xf = pf[0]
  yf = pf[1]
  angf = radians(pf[2])

  dxf = cos(angf)
  dyf = sin(angf)

  return [(xi - dxi, yi - dyi), (xf + dxf, yf + dyf)]


In [ ]:
# Desenha a reta

# Dado o x e o ang inicial

def desenha_reta(pi, pf, x_corte= -1):
  # pi = [x, y, ang, xinf, 'tipo']
  # pf = [x, -, -, -, 'r']
  xi = pi[0]
  yi = pi[1]
  angi = pi[2]
  xf = pf[0]

  if x_corte == -1:
    x = xf
  else:
    x = x_corte

  dy = tan(radians(angi)) * (x - xi)

  y = yi + dy

  return [x, y]


In [ ]:
# Desenha parábola

def desenha_parabola(pi, pf, x_corte= -1):
  # pi = [x, y, ang, -, 'tipo']
  # pf = [x, -, ang, -, 'tipo']
  xi = pi[0]
  yi = pi[1]
  tgi = tan(radians(pi[2]))
  xf = pf[0]
  tgf = tan(radians(pf[2]))

  a = (tgf - tgi) / (2*(xf - xi))
  b = tgi - 2*a*xi
  c = yi - (a*xi**2) - (b*xi)

  # yf = (a*xf**2) + (b*xf) + c

  if x_corte == -1:
    L = []
    x = xi

    passo = 0.1 # Segmentos de divisão da parábola

    quant_trechos = int((xf-xi)//passo)
    for i in range(quant_trechos): # -1
      passo_p = (xf-xi) / quant_trechos
      x += passo_p
      y = (a*x**2) + (b*x) + c
      L.append([x, y])

  else:
    x = x_corte
    y = (a*x**2) + (b*x) + c
    L = [x, y]

  return L


In [ ]:
# Desenha parábola com inflexão

def desenha_inflexao(pi, pf, x_corte= -1):
  # pi = [x, y, ang, -, 'tipo']
  # pf = [x, y, ang, xinf, 'tipo']
  # y1 = a.x1^2 + b.x1 + c
  # y2 = d.x2^2 + e.x2 + f

  xi = pi[0]
  yi = pi[1]
  angi = pi[2]
  xf = pf[0]
  yf = pf[1]
  angf = pf[2]
  xinf = pf[3]

  Dy = yf - yi
  Dx = xf - xi
  Dx1 = xinf - xi
  Dx2 = xf - xinf

  tgi = tan(radians(angi))
  tgf = tan(radians(angf))

  tginf = (2*Dy - Dx1*tgi - Dx2*tgf) / Dx

  a = (tginf - tgi) / (2 * Dx1)
  b = (xinf*tgi - xi*tginf) / Dx1
  c = yi - a*xi**2 - b*xi

  d = (tgf - tginf) / (2*Dx2)
  e = (xf*tginf - xinf*tgf) / Dx2
  f = yf - d*xf**2 - e*xf

  if x_corte == -1:
    x = xi

    passo = 0.1 # Segmentos de divisão da parábola

    L = []

    # Primeira parábola
    quant_trechos_1 = int((xinf-xi)//passo)
    for i in range(quant_trechos_1):
      passo1 = (xinf-xi) / quant_trechos_1
      x += passo1
      y = a*x**2 + b*x + c
      L.append([x, y])

    # Segunda parábola
    quant_trechos_2 = int((xf - xinf)//passo)
    for i in range(quant_trechos_2):
      passo2 = (xf-xinf) / quant_trechos_2
      x += passo2
      y = d*x**2 + e*x + f
      L.append([x, y])

  else:
    x = x_corte
    if x_corte <= xinf:
      y = a*x**2 + b*x + c
      L = [x, y]
    else:
      y = d*x**2 + e*x + f
      L = [x, y]

  return L


In [ ]:
# Cria um arquivo para cada cabo e um arquivo com todos os cabos

# Colocar os nomes nos cabos (criar dicionário e usar as chaves do dicionário)
# Corrigir as retas (em função do angulo e n da coordenada)


def desenha_cabos(dic_cabos, viga, orientacao): # orientação = "planta" ou "elevação"
  lista_cabos = list(dic_cabos.values()) # Lista de lista, sendo cada sublista as entradas de um cabo
  lista_nome_cabos = list(dic_cabos.keys())
  coordenadas_cabos=[] # Lista de lista de tuplas de todos os cabos, cada sublista tem as tuplas de um cabo

  # Trata cada cabo individual
  c = 0
  for lista_cabo_individual in lista_cabos:
    g = 0
    lista_cabo_ind_des = [] # Lista de tupla usada para gerar os cabos individuais, zera depois que é adicionado em coordenadas_cabos e gera o arquivo com o desenho
    for l in lista_cabo_individual:
      # Percorre cada coordenada
      # if lista_cabo_individual[g][-1] == 'r':
      if l[-1] == 'r':
        coord_fim = desenha_reta(lista_cabo_individual[g-1], lista_cabo_individual[g])
        # lista_cabo_individual[g-1][1] = coord_ini[1] # Ja tem q estar certo
        lista_cabo_individual[g][1] = coord_fim[1] # Adiciona o valor de y para o ponto final da reta
        lista_cabo_individual[g][2] = lista_cabo_individual[g-1][2]
        lista_cabo_ind_des.append(tuple(coord_fim))
      elif l[-1] == 'p': # O trecho p e p+1 é uma parábola
        coord_parab = desenha_parabola(lista_cabo_individual[g-1], lista_cabo_individual[g])
        lista_cabo_individual[g][1] = coord_parab[-1][1]
        for p in coord_parab:
          lista_cabo_ind_des.append(tuple(p))
      elif l[-1] == 'pi':
        coord_parab_inf = desenha_inflexao(lista_cabo_individual[g-1], lista_cabo_individual[g])
        for p in coord_parab_inf:
          lista_cabo_ind_des.append(tuple(p))
      else: # Primeiro ponto
        coord_inicial = [l[0], l[1]]
        lista_cabo_ind_des.append(tuple(coord_inicial))

      # lista_cabo_ind_des.append(l[0:2])
      g += 1
    # Insere os pontos das sobras de 1m para cada lado
    coord_sobras = sobras(lista_cabo_individual[0], lista_cabo_individual[-1])
    lista_cabo_ind_des.insert(0, coord_sobras[0])
    lista_cabo_ind_des.append(coord_sobras[1])

    # Gera desenho de cada cabo
    doc = ezdxf.new('R2010', setup=True) # create a new DXF drawing in R2010 fromat

    msp = doc.modelspace()

    doc.layers.add(name='CABO', color=5)
    doc.layers.add(name='FORMA', color=4)
    doc.layers.add(name='TEXTO', color=2)

    msp.add_polyline2d(viga, close=True, dxfattribs={'layer': 'FORMA'})
    msp.add_polyline2d(lista_cabo_ind_des, format='xy', dxfattribs={'layer': 'CABO'}) # add a 2D POLYLINE entity (ou msp.add_polyline3d() )
    raio = 0.32
    centro = tuple([lista_cabo_ind_des[0][0]-0.32, lista_cabo_ind_des[0][1]])
    msp.add_circle(tuple([lista_cabo_ind_des[0][0]-0.32, lista_cabo_ind_des[0][1]]), radius=raio, dxfattribs={'layer': 'CABO'})
    # msp.add_text("C." + str(c),  dxfattribs={'layer': 'TEXTO', 'height': 0.15}).set_placement(
    msp.add_text(lista_nome_cabos[c],  dxfattribs={'layer': 'TEXTO', 'height': 0.15}).set_placement(
        centro,
        align=TextEntityAlignment.MIDDLE_CENTER
    )

    doc.saveas(f'Cabo_{orientacao}_{lista_nome_cabos[c]}.dxf')

    coordenadas_cabos.append(lista_cabo_ind_des)
    c += 1
    # lista_cabo_ind_des = Lista de tuplas

  # Gera desenho de todos os cabos
  doc = ezdxf.new('R2010', setup=True) # create a new DXF drawing in R2010 fromat

  msp = doc.modelspace() # add new entities to the modelspace

  doc.layers.add(name='CABO', color=5)
  doc.layers.add(name='FORMA', color=4)
  doc.layers.add(name='TEXTO', color=2)

  msp.add_polyline2d(viga, close=True, dxfattribs={'layer': 'FORMA'})

  c = 0
  for cabo in coordenadas_cabos:
    msp.add_polyline2d(cabo, format='xyb', dxfattribs={'layer': 'CABO'}) # add a 2D POLYLINE entity (ou msp.add_polyline3d() )
    raio = 0.32
    centro = tuple([cabo[0][0]-0.32, cabo[0][1]])
    msp.add_circle(centro, radius=raio, dxfattribs={'layer': 'CABO'})
    msp.add_text(lista_nome_cabos[c],  dxfattribs={'layer': 'TEXTO', 'height': 0.15}).set_placement(
        centro,
        align=TextEntityAlignment.MIDDLE_CENTER
    )
    c += 1

  doc.saveas(f'Cabos_{orientacao}.dxf')
  # coordenadas_cabos = Lista de lista de tuplas


# Seções

In [ ]:
# # Desenha seção com os cabos
# # Corrigir diâmetro dos cabos (usuário tem q inserir)


# def desenha_seção(coordenadas_seção, dict_coordenadas_cabos):
#   texto_cabos = list(dict_coordenadas_cabos.keys())
#   coordenada_cabos = dict_coordenadas_cabos.values()

#   doc = ezdxf.new('R2010', setup=True)

#   msp = doc.modelspace()

#   doc.layers.add(name='CABO', color=5)
#   doc.layers.add(name='FORMA', color=4)
#   doc.layers.add(name='TEXTO', color=2)

#   msp.add_polyline2d(coordenadas_secao, close=True, dxfattribs={'layer': 'FORMA'})

#   c = 0
#   for i in coordenada_cabos:
#     msp.add_circle(i, radius=0.1, dxfattribs={'layer': 'CABO'})

#     d = -0.5*(-1)**c
#     centro = tuple([i[0]+d, i[1]])
#     msp.add_circle(centro, radius=0.32, dxfattribs={'layer': 'CABO'})
#     msp.add_text(texto_cabos[c],  dxfattribs={'layer': 'TEXTO', 'height': 0.15}).set_placement(
#         centro,
#         align=TextEntityAlignment.MIDDLE_CENTER
#     )
#     c += 1

#   doc.saveas('Seção_Cabos.dxf')


In [ ]:
# coordenadas_secao = [(0, 0), (-1.325, 0), (-2.05, 2.9), (-3.65, 3), (-3.65, 3.2), (0, 3.2), (0.0000, 2.95), (-1.441, 2.95), (-1.601, 2.7533), (-1.0252, 0.45), (-0.6252, 0.25), (0, 0.25)]
# coordenadas_cabos = {'C.1': (-1.8188, 2.8), 'C.2': (-1.6688, 2.2), 'C.3': (-1.5188, 1.6), 'C.4': (-1.3688, 1.0)}

# desenha_seção(coordenadas_secao, coordenadas_cabos)

# Seções desenho

In [ ]:
def gera_eixos_de_corte(vaos):
  eixos_de_corte = [0]
  for index, i in enumerate(vaos):
    dist_cortes = i/20
    for a in range(20):
      eixos_de_corte.append(eixos_de_corte[-1]+dist_cortes)
  return eixos_de_corte


In [ ]:
# vaos = [20, 30, 10]

# posicao_cortes = gera_eixos_de_corte(vaos)

# print(posicao_cortes)
# len(posicao_cortes)

In [ ]:
# Calcula as coordenadas para uma lista de posições,
# Calcula a posição dos cabos nos corte, mas só um sentido por vez (planta ou elevação)
# Precisa calcular os dois (planta e elevação) e juntar eles

# Precisa rodar a função do desenho longitudinal antes para corrigir os valores de y da lista de entrada antes

def gera_coord_cabos_planta(entrada, cortes):
  nomes_entrada = list(entrada.keys())

  coordenadas = {}

  for nome_cabo in nomes_entrada: # Olha cada cabo
    cabo = entrada[nome_cabo]
    coord_cabo_ind = []

    for index_corte, coord_x in enumerate(cortes):
      if coord_x < cabo[0][0]:
        pass
      else:
        c = 0
        # while coord_x > cabo[c][0] and coord_x <= cabo[-1][0]:
        while coord_x > cabo[c][0] and coord_x <= cabo[-1][0]:
          c += 1

        if cabo[c][-1] == '':
          posicao = list(cabo[c][:2])
        elif cabo[c][-1] == 'r':
          posicao = desenha_reta(cabo[c-1],cabo[c],coord_x)
        elif cabo[c][-1] == 'p':
          posicao = desenha_parabola(cabo[c-1],cabo[c],coord_x)
        elif cabo[c][-1] == 'pi':
          posicao = desenha_inflexao(cabo[c-1],cabo[c],coord_x)

        if coord_x > cabo[-1][0]:
          posicao = []

        coord_cabo_ind.append(posicao)

        for item in coord_cabo_ind:
          if item == []:
            coord_cabo_ind.remove([])

      coordenadas[nome_cabo] = coord_cabo_ind

  return coordenadas



In [ ]:
# Está igual para em planta

def gera_coord_cabos_elevacao(entrada, cortes):
  nomes_entrada = list(entrada.keys())

  coordenadas = {}

  for nome_cabo in nomes_entrada: # Olha cada cabo
    cabo = entrada[nome_cabo]
    coord_cabo_ind = []

    for index_corte, coord_x in enumerate(cortes):
      if coord_x < cabo[0][0]:
        pass
      else:
        c = 0
        # while coord_x > cabo[c][0] and coord_x <= cabo[-1][0]:
        while coord_x > cabo[c][0] and coord_x <= cabo[-1][0]:
          c += 1

        if cabo[c][-1] == '':
          posicao = list(cabo[c][:2])
        elif cabo[c][-1] == 'r':
          posicao = desenha_reta(cabo[c-1],cabo[c],coord_x)
        elif cabo[c][-1] == 'p':
          posicao = desenha_parabola(cabo[c-1],cabo[c],coord_x)
        elif cabo[c][-1] == 'pi':
          posicao = desenha_inflexao(cabo[c-1],cabo[c],coord_x)

        if coord_x > cabo[-1][0]:
          posicao = []

        coord_cabo_ind.append(posicao)

        for item in coord_cabo_ind:
          if item == []:
            coord_cabo_ind.remove([])

      coordenadas[nome_cabo] = coord_cabo_ind

  return coordenadas


In [ ]:
# # Junta os dicionários para juntar as coordenadas x e y

# dic_corte_planta = gera_coord_cabos_plano(entrada_planta, posicao_cortes)
# dic_corte_elevacao = gera_coord_cabos_plano(entrada_elevacao, posicao_cortes)

# print(dic_corte_planta)
# print(dic_corte_elevacao)

# # print(dic_corte_elevacao['C.1'])

In [ ]:
# Jusnta os dicionários com as coordenadas do corte em elevação e em planta e corrige o angulo


def junta_cortes (planta, elevacao, posicao_cortes, ang= 0):
  lista_coord_secoes = []
  for a in posicao_cortes:
    lista_coord_secoes.append({})

  nome_cabo = list(planta.keys())

  for cabo in nome_cabo:
    cortes_planta = planta[cabo]
    cortes_elevacao = elevacao[cabo]

    for c in list(range(len(cortes_planta))):
      posicao = posicao_cortes.index(cortes_planta[c][0])
      coord_ind = tuple([cortes_planta[c][1]-(cortes_elevacao[c][1]*tan(radians(ang))), cortes_elevacao[c][1]])
      lista_coord_secoes[posicao][cabo] = coord_ind

  return lista_coord_secoes

# SAIDA:
# lista_coord_secoes = [{'C1': (1, 2), 'C2': (1, 4)},
#                       {'C1': (2, 2), 'C2': (2, 4)},
#                       {'C1': (3, 2)},
#                       {'C2': (3, 3)}]


In [ ]:
# Melhorar o dxf, esta como lançamento de cabo adaptado

def desenha_secoes(dic_cabos, viga, vaos, diametro):
  eixos_de_corte = [0]
  for i in vaos:
    dist_cortes = i/20
    for a in range(20):
      eixos_de_corte.append(eixos_de_corte[-1]+dist_cortes)
  # eixos_de_corte = [0, 1, 2.5, 5]

  # lista_cabos = list(dic_cabos.values()) # lista de lista, sendo cada sublista as entradas de um cabo
  # lista_coord_secoes = [{'C1': (1, 2), 'C2': (1, 4)},
  #                       {'C1': (2, 2), 'C2': (2, 4)},
  #                       {'C1': (3, 2)},
  #                       {'C2': (3, 3)}]

  # lista_coord_secoes = dic_cabos

  lista_pos = []
  lista_nomes = []
  for index, pos in enumerate(eixos_de_corte):
    lista_pos.append(list(lista_coord_secoes[index].values()))
    lista_nomes.append(list(lista_coord_secoes[index].keys()))


  for index_secao, s in enumerate(eixos_de_corte):
    doc = ezdxf.new('R2010', setup=True) # create a new DXF drawing in R2010 fromat

    msp = doc.modelspace() # add new entities to the modelspace

    doc.layers.add(name='CABO', color=5)
    doc.layers.add(name='FORMA', color=4)
    doc.layers.add(name='TEXTO', color=2)

    msp.add_polyline2d(viga, close=True, dxfattribs={'layer': 'FORMA'})

    for c in range(len(lista_pos[index_secao])):
      # msp.add_polyline2d(cabo, format='xyb', dxfattribs={'layer': 'CABO'}) # add a 2D POLYLINE entity (ou msp.add_polyline3d() )
      # raio = 0.32
      raio = diametro/2
      centro = tuple([lista_pos[index_secao][c][0]+raio+0.15, lista_pos[index_secao][c][1]+raio])
      msp.add_circle(lista_pos[index_secao][c], radius=raio, dxfattribs={'layer': 'CABO'})
      msp.add_text(lista_nomes[index_secao][c],  dxfattribs={'layer': 'TEXTO', 'height': 0.15}).set_placement(
          centro,
          align=TextEntityAlignment.MIDDLE_CENTER
      )
    doc.saveas(f'Seção_{index_secao+1}.dxf')


# Interferência

In [ ]:
def matriz_distancias(nomes, posicoes):
    # Cria uma matriz de distâncias entre todos os cabos fornecidos.

    n = len(posicoes)
    matriz_distancias = [[0] * n for c in range(n)]

    for i in range(n):
        for j in range(n):
            if i != j:
                matriz_distancias[i][j] = sqrt((posicoes[j][0] - posicoes[i][0]) ** 2 + (posicoes[j][1] - posicoes[i][1]) ** 2)

    matriz_calc = pd.DataFrame(matriz_distancias, columns=[f'{i}' for i in nomes], index=[f'{i}' for i in nomes])
    return matriz_calc



In [ ]:
# Verifica interferência entre os cabos

def verifica_interferencia (pos_inicial, pos_final, passo, diametro_bainha, entrada_planta, entrada_elevacao):
  lista_geral = []
  # lista_interfrência = []

  lista_secoes = []
  num_secoes = int((pos_final - pos_inicial) // passo)

  for s in range(num_secoes+1):
    lista_secoes.append(pos_inicial + s*passo)
  lista_secoes.append(pos_final)

  dic_corte_planta = gera_coord_cabos_planta(entrada_planta, lista_secoes)
  dic_corte_elevacao = gera_coord_cabos_elevacao(entrada_elevacao, lista_secoes)

  lista_coord_secoes = junta_cortes(dic_corte_planta, dic_corte_elevacao, lista_secoes, angulo_alma)

  for index, secao in enumerate(lista_secoes):
    lista_pos_cabos = lista_coord_secoes[index]
    lista_nomes = list(lista_pos_cabos.keys())
    lista_posicoes = list(lista_pos_cabos.values())
    lista_geral.append(matriz_distancias(lista_nomes, lista_posicoes))


  return lista_secoes, lista_geral



#Entrada

In [ ]:
# Entrada de dados:

# O ponto base de todas as entradas é o centro inferior da seção (base do eixo de simetria)

# [x, y, anglo, xinf, tipo]
# tipos:
# - "": primeiro ponto
# - "r": reta
# - "p": parábola
# - "pi": parábola com inflexão

entrada_planta = {'C.1': [[0,-0.4938,-4.2836,0,''], [0,0,1,0,'r'], [3,0,0,0,'p'], [20.5,0,0,0,'r'], [40.5,0.0424,0,30.5,'pi'], [57.1,0,0,0,'r'], [77.1,-0.6438,0,67.1,'pi'], [94.6,0,0,0,'r'], [96.6,0,4.2836,0,'p'], [97.6,0,0,0,'r']],
                  'C.2': [[0,0,0,0,''], [39.2,0,0,0,'r']]}

entrada_elevacao = {'C.1': [[0, 2.8, 7.6804, 0, ''], [1,0,0,0,'r'], [3,0,0,0,'p'], [20.5,0,0,0,'r'], [40.5,0.33,0,30.5,'pi'], [57.1,0,0,0,'r'], [77.1,3.07,0,67.1,'pi'], [94.6,0,0,0,'r'], [96.6,0, -7.6804,0,'p'], [97.6,0,0,0,'r']],
                    'C.2': [[0,0.925,-7.861,0,''], [1,0,0,0,'r'], [16.1,0,0,0,'p'], [23.1,0,0,0,'r'], [38.2,0,7.861,0,'p'], [39.2,0,0,0,'r']]}

angulo_alma = 0  # Ângulo em graus (0 - vertical e 90 - horizontal)
diametro_bainha = 0.1  # Em metros

vaos = [16.25, 65.1, 16.25]
secao = [(0, 0), (-1.325, 0), (-2.05, 2.9), (-3.65, 3), (-3.65, 3.2), (0, 3.2), (0.0000, 2.95), (-1.441, 2.95), (-1.601, 2.7533), (-1.0252, 0.45), (-0.6252, 0.25), (0, 0.25)]



In [ ]:

# Roda as funções para criar os desenhos

comp_total = 0
for c in vaos:
  comp_total += c

altura_viga = 0
for h in secao:
  if h[1] >altura_viga:
    altura_viga = h[1]


viga = [(0, 0), (comp_total, 0), (comp_total, altura_viga), (0, altura_viga)]

desenha_cabos(entrada_planta, viga, 'planta')
desenha_cabos(entrada_elevacao, viga, 'elevação')

posicao_cortes = gera_eixos_de_corte(vaos)

dic_corte_planta = gera_coord_cabos_planta(entrada_planta, posicao_cortes)
dic_corte_elevacao = gera_coord_cabos_elevacao(entrada_elevacao, posicao_cortes)

lista_coord_secoes = junta_cortes(dic_corte_planta, dic_corte_elevacao, posicao_cortes, angulo_alma)

desenha_secoes(lista_coord_secoes, secao, vaos, diametro_bainha)

In [ ]:
# Entrada verificação


pos_inicial = 10
pos_final = 50
passo = 5



In [ ]:
# Roda as funções para verificar interferência e printar

lista_secoes, lista_matrizes = verifica_interferencia (pos_inicial, pos_final, passo, diametro_bainha, entrada_planta, entrada_elevacao)

for index, i in enumerate(lista_secoes):
  print(f'Posição: {i}')
  print(lista_matrizes[index])



Posição: 10
          C.1       C.2
C.1  0.000000  3.212777
C.2  3.212777  0.000000
Posição: 15
          C.1       C.2
C.1  0.000000  3.374548
C.2  3.374548  0.000000
Posição: 20
         C.1      C.2
C.1  0.00000  3.37999
C.2  3.37999  0.00000
Posição: 25
          C.1       C.2
C.1  0.000000  3.079101
C.2  3.079101  0.000000
Posição: 30
          C.1       C.2
C.1  0.000000  1.897313
C.2  1.897313  0.000000
Posição: 35
          C.1       C.2
C.1  0.000000  0.356824
C.2  0.356824  0.000000
Posição: 40
     C.1
C.1    0
Posição: 45
     C.1
C.1    0
Posição: 50
     C.1
C.1    0
Posição: 50
Empty DataFrame
Columns: []
Index: []


#Rascunho

In [ ]:

# perfil = [(.5, 0), (.5, 2), (-.5, 2), (-.5, 0)]
# vaos = [20, 10, 30]

# desenha_secoes(lista_coord_secoes, perfil, vaos)

In [ ]:
# Cria um arquivo para cada corte

# def desenha_cortes(dic_cabos, viga, vaos):
#   eixos_de_corte = [0]
#   for i in vaos:
#     dist_cortes = i/20
#     for a in range(20):
#       eixos_de_corte.append(eixos_de_corte[-1]+dist_cortes)

#   # Varer pela coordenada, n pelo tipo

#   lista_cabos = list(dic_cabos.values())
#   coordenadas_cabos_des = [] # Lista de lista de tuplas dos cabos

#   # Trata cada cabo individual
#   c = 1
#   for lista_cabo_individual in lista_cabos:
#     g = 0
#     lista_coord_ind = []
#     for l in lista_cabo_individual:
#       # Percorre cada coordenada
#       # if lista_cabo_individual[g][-1] == 'r':
#       if l[-1] == 'r':
#         coord_fim = desenha_reta(lista_cabo_individual[g-1], lista_cabo_individual[g])
#         # lista_cabo_individual[g-1][1] = coord_ini[1] # Ja tem q estar certo
#         lista_cabo_individual[g][1] = coord_fim[1] # Adiciona o valor de y para o ponto final da reta
#         lista_cabo_individual[g][2] = lista_cabo_individual[g-1][2]
#         lista_coord_ind.append(tuple(coord_fim))
#       elif l[-1] == 'p': # O trecho p e p+1 é uma parábola
#         coord_parab = desenha_parabola(lista_cabo_individual[g-1], lista_cabo_individual[g])
#         lista_cabo_individual[g][1] = coord_parab[-1][1]
#         for p in coord_parab:
#           lista_coord_ind.append(tuple(p))
#       elif l[-1] == 'pi':
#         coord_parab_inf = desenha_inflexao(lista_cabo_individual[g-1], lista_cabo_individual[g])
#         for p in coord_parab_inf:
#           lista_coord_ind.append(tuple(p))
#       else: # Primeiro ponto
#         coord_inicial = [l[0], l[1]]
#         lista_coord_ind.append(tuple(coord_inicial))

#       # lista_coord_ind.append(l[0:2])
#       g += 1
#     coord_sobras = sobras(lista_cabo_individual[0], lista_cabo_individual[-1])
#     lista_coord_ind.insert(0, coord_sobras[0])
#     lista_coord_ind.append(coord_sobras[1])

#     # Gera desenho de cada cabo
#     doc = ezdxf.new('R2010', setup=True) # create a new DXF drawing in R2010 fromat

#     msp = doc.modelspace()

#     doc.layers.add(name='CABO', color=5)
#     doc.layers.add(name='FORMA', color=4)
#     doc.layers.add(name='TEXTO', color=2)

#     msp.add_polyline2d(viga, close=True, dxfattribs={'layer': 'FORMA'})
#     msp.add_polyline2d(lista_coord_ind, format='xy', dxfattribs={'layer': 'CABO'}) # add a 2D POLYLINE entity (ou msp.add_polyline3d() )
#     raio = 0.32
#     centro = tuple([lista_coord_ind[0][0]-0.32, lista_coord_ind[0][1]])
#     msp.add_circle(tuple([lista_coord_ind[0][0]-0.32, lista_coord_ind[0][1]]), radius=raio, dxfattribs={'layer': 'CABO'})
#     msp.add_text("C." + str(c),  dxfattribs={'layer': 'TEXTO', 'height': 0.15}).set_placement(
#         centro,
#         align=TextEntityAlignment.MIDDLE_CENTER
#     )

#     doc.saveas(f'Cabo_{c}.dxf')

#     coordenadas_cabos_des.append(lista_coord_ind)
#     c += 1
#     # lista_coord_ind = Lista de tuplas

#   # Gera desenho de todos os cabos
#   doc = ezdxf.new('R2010', setup=True) # create a new DXF drawing in R2010 fromat

#   msp = doc.modelspace() # add new entities to the modelspace

#   doc.layers.add(name='CABO', color=5)
#   doc.layers.add(name='FORMA', color=4)
#   doc.layers.add(name='TEXTO', color=2)

#   msp.add_polyline2d(viga, close=True, dxfattribs={'layer': 'FORMA'})

#   c = 1
#   for cabo in coordenadas_cabos_des:
#     msp.add_polyline2d(cabo, format='xyb', dxfattribs={'layer': 'CABO'}) # add a 2D POLYLINE entity (ou msp.add_polyline3d() )
#     raio = 0.32
#     centro = tuple([cabo[0][0]-0.32, cabo[0][1]])
#     msp.add_circle(centro, radius=raio, dxfattribs={'layer': 'CABO'})
#     msp.add_text("C." + str(c),  dxfattribs={'layer': 'TEXTO', 'height': 0.15}).set_placement(
#         centro,
#         align=TextEntityAlignment.MIDDLE_CENTER
#     )
#     c += 1

#   doc.saveas(f'Cabos.dxf')


In [ ]:
# entrada_planta = {'C.1': [[0, 2.695, -16.3271, 0, ''], [1, 0, 0, 0, 'r'], [17.4, 0, 0, 0, 'p'], [47.4, 0, 0, 0, 'r'], [52.6, 0, -1.9377, 0, 'p'], [55.6, 0, 0, 0, 'p'], [60, 0, 0, 0, 'r']],
#                   'C.2': [[4, 2, -7.15, 0, ''], [6, 0, 0, 0, 'r'], [11, 0, 10, 0, 'p'], [13, 0, 0, 0, 'r'], [18, 1.8, 10, 12, 'pi'], [21, 0, 0, 0, 'r']]}

# entrada_elevacao = {'C.1': [[0, 2.695, -16.3271, 0, ''], [1, 0, 0, 0, 'r'], [17.4, 0, 0, 0, 'p'], [47.4, 0, 0, 0, 'r'], [52.6, 0, -1.9377, 0, 'p'], [55.6, 0, 0, 0, 'p'], [60, 0, 0, 0, 'r']],
#                     'C.2': [[4, 2, -7.15, 0, ''], [6, 0, 0, 0, 'r'], [11, 0, 10, 0, 'p'], [13, 0, 0, 0, 'r'], [18, 1.8, 10, 12, 'pi'], [21, 0, 0, 0, 'r']]}

# secao = [(0, 0), (-1.325, 0), (-2.05, 2.9), (-3.65, 3), (-3.65, 3.2), (0, 3.2), (0.0000, 2.95), (-1.441, 2.95), (-1.601, 2.7533), (-1.0252, 0.45), (-0.6252, 0.25), (0, 0.25)]
# vaos = [20, 10, 30]

# desenha_cortes(entrada_planta, secao, vaos)


In [ ]:
# RASCUNHO!!!

# Gera a coordenada dos cabos de cada seção
def gera_coord_cabos_secao(dic_cabos_planta, dic_cabos_elevacao, eixos_de_corte):
  lista_nome_cabos = list(dic_cabos_planta.keys()) # Lista com o nome dos cabos
  lista_entrada_cabos_planta = list(dic_cabos_planta.values()) # Lista das listas com as cordenadas em planta de todos os cabos
  lista_entrada_cabos_elevacao = list(dic_cabos_elevacao.values()) # Lista das listas com as cordenadas em elevação de todos os cabos
  lista_coordenadas_secoes_planta = {}
  lista_coordenadas_secoes_elevacao = {}
  # coordenadas_x = eixos_de_corte
  coordenadas_x = []
  for eixo in eixos_de_corte:
    coordenadas_x.append(eixo)

  # Olhando comportamento em planta
  for index, lista_entrada_individual in enumerate(lista_entrada_cabos_planta):
    # Analisa cabo por cabo
    for entrada in lista_entrada_individual:
      # Cada cabo
      coordenadas_x.append(entrada[0])
    coordenadas_x.sort() # Ordena a lista em ordem crescente
    lista_coord_corte = list(dict.fromkeys(coordenadas_x)) # Remove duplicados, mantendo a ordem existente
    e = 0
    lista_indiv = []
    # for index_corte, x in enumerate(lista_coord_corte):
    print(lista_coord_corte, 2)
    for index_corte, x in enumerate(lista_coord_corte):
      cabo = lista_nome_cabos[index] # Guarda o nome do cabo em análise
      while x > lista_entrada_individual[e][0] and e < len(lista_entrada_individual)-1:
        # Descobre antes de qual entrada a seção está
        e += 1
      # if x == 0:
      #   # Primeiro ponto
      #   coord = list([0, lista_entrada_individual[e][1]])
      #   lista_indiv.append(coord)
      # elif x <= lista_entrada_individual[e][0]:
      if x <= lista_entrada_individual[e][0]:
        # Ou seja x < lista_entrada_individual[e][0] and x > lista_entrada_individual[1][0]:
        if lista_entrada_individual[e][-1] == '':
          coord = list([0, lista_entrada_individual[e][1]])
        elif lista_entrada_individual[e][-1] == 'r':
          coord = list(desenha_reta(lista_entrada_individual[e-1], lista_entrada_individual[e], x))
        elif lista_entrada_individual[e][-1] == 'p':
          coord = list(desenha_parabola(lista_entrada_individual[e-1], lista_entrada_individual[e], x))
        elif lista_entrada_individual[e][-1] == 'pi':
          coord = list(desenha_inflexao(lista_entrada_individual[e-1], lista_entrada_individual[e], x))
        lista_indiv.append(coord)
      else:
        pass
    lista_coordenadas_secoes_planta[cabo] = lista_indiv
    lista_indiv = []
  print(lista_coordenadas_secoes_planta)


# Entrada:
dic_cabos_elevacao = {'C.1': [[0, 3.695, 0, 0, ''], [10, 0, 0, 0, 'r'], [66, 0, 0, 0, 'r']],
                      'C.2': [[0, 2, 0, 0, ''], [16, 0, 0, 0, 'r']]}
dic_cabos_planta = {'C.1': [[0, 2, -7.15, 0, ''], [2, 0, 0, 0, 'r'], [7, 0, 10, 0, 'p'], [9, 0, 0, 0, 'r'], [14, 1.8, 10, 12, 'pi'], [16, 0, 0, 0, 'r']],
                    'C.2': [[0, 2.695, -16.3271, 0, ''], [1, 0, 0, 0, 'r'], [17.4, 0, 0, 0, 'p'], [47.4, 0, 0, 0, 'r'], [52.6, 0, -1.9377, 0, 'p'], [55.6, 0, 0, 0, 'p'], [60, 0, 0, 0, 'r']]}

eixos_de_corte = [0, 0.5, 1, 2.5, 5, 40, 66]

gera_coord_cabos_secao(dic_cabos_planta, dic_cabos_elevacao, eixos_de_corte)

# No final eu tenho q ter 1 dicionário para cada cabo onde as chaves são a
# posição e os valores as coordenadas x e y


# Saida:
# lista_coord_secoes = [{'C1': (1, 2), 'C2': (1, 4)},
#                       {'C1': (2, 2), 'C2': (2, 4)},
#                       {'C1': (3, 2)},
#                       {'C2': (3, 3)}]

In [ ]:
# Desenha os cabos

# Precisa desenhar a forma tbm
# Mudar, se arco para trechos de reta descrevendo uma parábola (c/ 50 cm)
# ajustar o final da parábola, sem dar o x, só o ângulo
# Criar arquivos separados para cada cabo e um com tudo junto

def desenha_cabo_separado(dados):
  dados.append([0])
  coordenadas_tuplas = []
  for i in range(len(dados)-1):
    if dados[i+1][-1] == 'p':
      c1 = dados[i]
      c2 = dados[i+1]

      dx = c2[0] - c1[0]
      dy = c2[1] - c1[1]

      tg = dy / dx

      if c1[2] == 0:
        x = 0
      else:
        x = dx

      r = abs(dy/2) + dx/(2*tg) - x/tg

      corda = sqrt(dx**2 + dy**2)

      h = sqrt(r**2 - (corda/2)**2)

      b = (r - h) / (corda/2)

      dados[i].insert(2, b) # Indicação de q vai ser arco deve vir no primeiro ponto
    else:
      # Se for reta b=0
      dados[i].insert(2, 0)
    dados[i].pop(-1)
    dados[i].pop(-1)
    coordenadas_tuplas.append(tuple(dados[i]))

  doc = ezdxf.new('R2010') # create a new DXF drawing in R2010 fromat

  msp = doc.modelspace() # add new entities to the modelspace
  msp.add_polyline2d(coordenadas_tuplas, format='xyb') # add a 2D POLYLINE entity (ou msp.add_polyline3d() )
  doc.saveas(f'Cabo_{}.dxf')

  # return lista_cabos.append()

K = [[0.000, 2.800, 7.6804, 'r'], [1.000, 1, 7.6804, 'r'], [3.000, 0, 0, 'p'], [10, 0, 0, 'r'], [15, 2, 5, 'p'], [16, 2.5, 5, 'r']]
desenha_cabo_separado(K)

def desenha_cabo_junto(dic_cabos):
  lista_cabo = list(dic_cabos.values)


In [ ]:
# Cria um dxf com uma polyline
# Está limitado para uma das extremidades serem 0
# As coordenadas x tem q estar certas
# Falta parabola com inflexão

c1 = [5, 2, 1]
c2 = [10, 0, 0]

dx = c2[0] - c1[0]
dy = c2[1] - c1[1]

tg = dy / dx

p = [dx/2, dy/2]

if c1[2] == 0:
  x = 0
else:
  x = dx

r = abs(dy/2) + dx/(2*tg) - x/tg

corda = sqrt(dx**2 + dy**2)

h = sqrt(r**2 - (corda/2)**2)

b = (r - h) / (corda/2)

c1.pop(-1)
c1.append(b)  # Invertendo sinal de b inverte o lado da curva (consequentemente o lado q o angulo é 0)

a1 = tuple(c1)
a2 = tuple(c2)

s = [a1, a2]

# Uma função para cada (cabos, seções)
def gerar_dxf(coordenadas):
  doc = ezdxf.new('R2010') # create a new DXF drawing in R2010 fromat

  msp = doc.modelspace() # add new entities to the modelspace
  msp.add_polyline2d(coordenadas, format='xyb') # add a 2D POLYLINE entity (ou msp.add_polyline3d() )
  doc.saveas('test_polyline_2d.dxf')

gerar_dxf(s)


# Importa CSV

In [ ]:
import csv

In [ ]:

lista_csv = []

cabos_planta = {}
cabos_elevacao = {}

with open('Com Espaco.CSV', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';', quotechar='|')
    for row in spamreader:
        lista_csv.append(row)
    print(lista_csv)
    lista_csv.pop(0)
    lista_csv.pop(0)
    print(lista_csv)
    lista_temp = []
    for linha in lista_csv:
      if linha[0] != '':
        print(linha)



In [ ]:
# Importar direto de um csv

import numpy as np
import pandas as pd

df_csv = pd.read_csv("teste.csv", sep=";") # O padrão é ','

df_csv

In [ ]:
df_n = pd.read_csv('novo.csv')

df_n


# Salva CSV

In [ ]:
import csv

# talvez tenha q varrer a lista e ir escrevendo aos poucos

with open('teste.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=';')
    spamwriter.writerow(s) # Cada tupla por célula


In [ ]:
# Gera um csv para salvar

L = [[1, 2, 3], [4, 5, 6], [7, 8]]

arquivo = open("novo.csv", "w", newline="")
for i in range(len(L)):
  arquivo.write('teste'+'\n')
arquivo.close()


In [ ]:

L = [[1, 2, 3], [4, 5, 6], [7, 8]]

novo = pd.DataFrame(L)
novo.to_csv('novo.csv')


# Rascunho

In [ ]:
# Exemplo de como criar cota

# Crie um novo documento DXF
doc = ezdxf.new(dxfversion='R2010')
msp = doc.modelspace()

# Adicione uma linha de exemplo
start_point = (0, 0)
end_point = (0, 10)
loc_point = (-2, 0)
msp.add_line(start_point, end_point)

# Adicione uma cota linear
msp.add_aligned_dim(
    p1=start_point,
    p2=end_point,
    distance=1,
    override={
        "dimtad": 1
        }
).render()

# Salve o arquivo DXF
doc.saveas('cotas_exemplo.dxf')


In [ ]:
# N lembro oq é

def desenha_cabos(dic_cabos, viga):
  for cabo in
  lista_cabos = list(dic_cabos.values())
  coordenadas_cabos=[] # Lista de tuplas de todos os cabos



  # Trata cada cabo individual
  c = 1
  for i in lista_cabos:
    lista_cabo_individual = i # Dá para substituir o i dereto pela 'lista_cabo_individual'?

    g = 0
    lista_cabo_ind_des = []
    for l in lista_cabo_individual:
      # Percorre cada coordenada
      if lista_cabo_individual[g][-1] == 'r':
        coord_ini, coord_fim = desenha_reta(lista_cabo_individual[g-1], lista_cabo_individual[g])
        lista_cabo_ind_des.append(coord_ini)
        lista_cabo_ind_des.append(coord_fim)
      elif lista_cabo_individual[g][-1] == 'p': # O trecho p e p+1 é uma parábola
        coord_parab = desenha_parabola(lista_cabo_individual[g-1], lista_cabo_individual[g])
        for s in coord_parab:
          lista_cabo_ind_des.append(s)
      elif lista_cabo_individual[g][-1] == 'pi':
        # Criar função para parabola com inflexão
        pass

      lista_cabo_ind_des.append(l[0:2])
      g += 1

    # Transforma para lista de tuplas
    coordenadas_tuplas = []
    for a in lista_cabo_ind_des:
      coordenadas_tuplas.append(tuple(a))

    doc = ezdxf.new('R2010', setup=True) # create a new DXF drawing in R2010 fromat

    msp = doc.modelspace() # add new entities to the modelspace

    doc.layers.add(name='CABO', color=5)
    doc.layers.add(name='FORMA', color=4)
    doc.layers.add(name='TEXTO', color=2)

    msp.add_polyline2d(viga, close=True, dxfattribs={'layer': 'FORMA'})
    msp.add_polyline2d(coordenadas_tuplas, format='xy', dxfattribs={'layer': 'CABO'}) # add a 2D POLYLINE entity (ou msp.add_polyline3d() )
    doc.saveas(f'Cabo_{c}.dxf')

    coordenadas_cabos.append(coordenadas_tuplas)
    c += 1

  # Gera desenho de todos os cabos
  doc = ezdxf.new('R2010', setup=True) # create a new DXF drawing in R2010 fromat

  msp = doc.modelspace() # add new entities to the modelspace

  doc.layers.add(name='CABO', color=5)
  doc.layers.add(name='FORMA', color=4)
  doc.layers.add(name='TEXTO', color=2)

  msp.add_polyline2d(viga, close=True, dxfattribs={'layer': 'FORMA'})
  for cabo in coordenadas_cabos:
    msp.add_polyline2d(cabo, format='xyb', dxfattribs={'layer': 'CABO'}) # add a 2D POLYLINE entity (ou msp.add_polyline3d() )
  doc.saveas(f'Cabos.dxf')

SyntaxError: invalid syntax (<ipython-input-31-978948e180cb>, line 4)